In [1]:
# imports
import sys
import os
import socket
import pandas as pd
import glob
from datetime import datetime

In [2]:
# read in test diagnosis file
dementia_dx_test = pd.read_parquet(os.getcwd() + '/dementia_patients_test_data.parquet')
dementia_dx_test.head()

,e_patid,eventdate,sysdate,constype,consid,medcode,enttype,data1,data2,data3,data4,data5,data6,data7,data8
506109,157719246,2001-07-19,2002-02-07,2,1115667,1350,288,0.0,NaN,0.0,0,NaN,NaN,0.0,None
506110,157719246,2001-07-19,2002-02-07,2,1115667,1350,288,0.0,NaN,0.0,0,NaN,NaN,0.0,None
506115,157719246,2001-07-19,2002-02-07,2,1115667,1350,288,0.0,NaN,0.0,0,NaN,NaN,0.0,None
506116,157719246,2001-07-19,2002-02-07,2,1115667,1350,288,0.0,NaN,0.0,0,NaN,NaN,0.0,None
506118,157719246,2001-07-19,2002-02-07,2,1115667,1350,288,0.0,NaN,0.0,0,NaN,NaN,0.0,None


In [3]:
# read in referral diagnosis file
dementia_dx_ref = pd.read_parquet(os.getcwd() + '/dementia_patients_referral_data.parquet')
dementia_dx_ref.head()

,e_patid,eventdate,sysdate,constype,consid,medcode,nhsspec,attendance,urgency
3074,10525191,2003-05-02,2003-05-02,4,1227502,6578,54,1,2
3079,10525191,2003-07-01,2003-07-01,4,1227532,6578,0,0,2
6221,11155213,2009-10-01,2009-10-02,4,805469,1916,70,0,2
6738,112629220,2006-05-03,2006-05-04,4,1845552,1350,54,1,4
9910,117980213,2014-03-07,2014-03-10,4,2482894,1350,41,0,2


In [4]:
# get unique e_patids from both
e_patids_test = dementia_dx_test['e_patid'].unique()
e_patids_ref = dementia_dx_ref['e_patid'].unique()

In [5]:
# Replace nan eventdates with sysdates and make date object
dementia_dx_test['eventdate'].fillna(dementia_dx_test['sysdate'], inplace = True)
dementia_dx_ref['eventdate'].fillna(dementia_dx_ref['sysdate'], inplace = True)

In [6]:
# format event date
dementia_dx_test['eventdate'] = pd.to_datetime(
    dementia_dx_test['eventdate'], format = "%d/%m/%Y")
dementia_dx_ref['eventdate'] = pd.to_datetime(
    dementia_dx_ref['eventdate'], format = "%d/%m/%Y")

In [7]:
# get first dx date in test for each e_patid
first_dementia_code_dates_test = []
for i in e_patids_test:
    first_dementia_code_dates_test += [min(
        dementia_dx_test[dementia_dx_test['e_patid'] == i]['eventdate'])]

In [8]:
# make df of dates
df_first_dates_test = pd.DataFrame({"e_patid": e_patids_test, "eventdate": first_dementia_code_dates_test})

In [9]:
df_first_dates_test.head()

,e_patid,eventdate
0,157719246,2001-07-19
1,636522123,2017-07-31


In [10]:
# merge with dx test df to get first dx row in test
first_dementia_dx_test = df_first_dates_test.merge(
    right = dementia_dx_test, on = ['e_patid', 'eventdate'], how = 'inner')

In [11]:
first_dementia_dx_test.shape[0]

6

In [12]:
dementia_dx_test.shape[0]

6

In [13]:
# save as parquet
first_dementia_dx_test.to_parquet(os.getcwd() + '/first_dementia_dx_in_primary_test.parquet')

In [14]:
# get first dx date in referral for each e_patid
first_dementia_code_dates_ref = []
for i in e_patids_ref:
    first_dementia_code_dates_ref += [min(
        dementia_dx_ref[dementia_dx_ref['e_patid'] == i]['eventdate'])]

In [15]:
# make df
df_first_dates_ref = pd.DataFrame({"e_patid": e_patids_ref, "eventdate": first_dementia_code_dates_ref})

In [16]:
df_first_dates_ref.head()

,e_patid,eventdate
0,10525191,2003-05-02
1,11155213,2009-10-01
2,112629220,2006-05-03
3,117980213,2014-03-07
4,123982213,1998-08-05


In [17]:
# merge with dx file to get first dx file for each e_patid
first_dementia_dx_ref = df_first_dates_ref.merge(
    right = dementia_dx_ref, on = ['e_patid', 'eventdate'], how = 'inner')

In [18]:
first_dementia_dx_ref.shape[0]

4549

In [19]:
dementia_dx_ref.shape[0]

5243

In [20]:
# save as parquet
first_dementia_dx_ref.to_parquet(os.getcwd() + '/first_dementia_dx_in_primary_referral.parquet')